In [12]:
# ライブラリのインポート
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
from torch import nn,optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import transforms
from torchinfo import summary
from torch.autograd import Variable

In [13]:
# df = pd.read_csv("https://raw.githubusercontent.com/aweglteo/tokyo_weather_data/main/data.csv", parse_dates=True, index_col=0)
# df.to_csv("./data/wether.csv")
df = pd.read_csv("./data/wether.csv", parse_dates=True, index_col=0)
df

,cloud,wind,ave_tmp,max_tmp,min_tmp,rain
2018-05-17,9.0,3.7,24.0,28.4,21.4,0.0
2018-05-18,7.8,3.0,23.4,27.1,20.0,0.0
2018-05-19,7.5,3.9,21.5,26.3,15.3,0.0
2018-05-20,4.3,3.5,16.8,22.1,13.1,0.0
2018-05-21,10.0,2.4,20.3,27.2,13.7,0.0
...,...,...,...,...,...,...
2023-05-13,10.0,3.0,16.7,18.1,15.4,5.0
2023-05-14,10.0,3.6,16.9,21.2,14.3,3.0
2023-05-15,9.8,2.0,16.1,18.1,14.6,15.5
2023-05-16,2.8,2.6,19.9,27.5,13.1,0.0


In [14]:
def normalize(r):
    norm = np.linalg.norm(r)
    return r / norm

In [15]:
cloud = normalize(df["cloud"])
wind = normalize(df["wind"])
ave_tmp = normalize(df["ave_tmp"])
max_tmp = normalize(df["max_tmp"])
min_tmp = normalize(df["min_tmp"])
rain = normalize(df["rain"])

data = pd.DataFrame(
    np.array([
        cloud,
        wind,
        ave_tmp,
        max_tmp,
        min_tmp,
        rain        
    ]).T,
    columns=[
        "cloud",
        "wind",
        "ave_tmp",
        "max_tmp",
        "min_tmp",
        "rain"
    ]
)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cloud    1827 non-null   float64
 1   wind     1827 non-null   float64
 2   ave_tmp  1827 non-null   float64
 3   max_tmp  1827 non-null   float64
 4   min_tmp  1827 non-null   float64
 5   rain     1827 non-null   float64
dtypes: float64(6)
memory usage: 85.8 KB


In [16]:
data

,cloud,wind,ave_tmp,max_tmp,min_tmp,rain
0,0.028303,0.029548,0.030535,0.029352,0.032809,0.000000
1,0.024529,0.023958,0.029772,0.028008,0.030662,0.000000
2,0.023586,0.031145,0.027354,0.027181,0.023457,0.000000
3,0.013522,0.027951,0.021375,0.022841,0.020084,0.000000
4,0.031448,0.019166,0.025828,0.028111,0.021004,0.000000
...,...,...,...,...,...,...
1822,0.031448,0.023958,0.021247,0.018707,0.023610,0.007955
1823,0.031448,0.028749,0.021502,0.021910,0.021923,0.004773
1824,0.030819,0.015972,0.020484,0.018707,0.022383,0.024659
1825,0.008805,0.020763,0.025319,0.028422,0.020084,0.000000


In [17]:
df_train, df_test = train_test_split(data, test_size=0.2, shuffle=False)

window_size = 20
n_data = len(df) - window_size

n_dim = df_train.shape[1]
n_train = len(df_train) - window_size
n_test = len(df_test) -window_size


# 正解データを準備
train = np.array([df_train[i:i+window_size] for i in range(len(df_train) - window_size)])
train_labels = np.array([df_train[i+window_size] for i in range(len(df_train) - window_size)])

# テストデータを準備
test = np.array([df_test[i:i+window_size] for i in range(len(df_test) - window_size)])
test_labels = np.array([df_test[i+window_size] for i in range(len(df_test) - window_size)])


6